In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import fasttext
from khaiii import KhaiiiApi
from tqdm.notebook import tqdm
# pd.set_option('display.max_rows', None)

In [2]:
song = pd.read_json("res/song_meta.json")[["id", "song_name", "artist_id_basket", "artist_name_basket",\
                                            "album_id", "album_name", "song_gn_gnr_basket",\
                                            "song_gn_dtl_gnr_basket", "issue_date"]]
genre = pd.read_json("res/genre_gn_all.json", typ="series")
train = pd.read_json("res/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
val = pd.read_json("res/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
test = pd.read_json("res/test.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
display(song, genre, train, val, test)

,id,song_name,artist_id_basket,artist_name_basket,album_id,album_name,song_gn_gnr_basket,song_gn_dtl_gnr_basket,issue_date
0,0,Feelings,[2727],[Various Artists],2255639,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,[GN0900],[GN0901],20140512
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[29966],[Murray Perahia],376431,"Bach : Partitas Nos. 2, 3 & 4",[GN1600],"[GN1601, GN1606]",20080421
2,2,Solsbury Hill (Remastered 2002),[3361],[Peter Gabriel],4698747,Hit,[GN0900],[GN0901],20180518
3,3,Feeling Right (Everything Is Nice) (Feat. Popc...,[838543],[Matoma],2644882,Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],"[GN1102, GN1101]",20151016
4,4,그남자 그여자,[560160],[Jude Law],2008470,그남자 그여자,[GN1800],"[GN1802, GN1801]",20110824
...,...,...,...,...,...,...,...,...,...
707984,707984,Coffin For Head Of State,[166499],[Fela Kuti],65254,The Best Best Of The Black President,[GN2000],[GN2001],19991219
707985,707985,Change Of Heart,[11837],[Cyndi Lauper],44141,True Colors,[GN0900],[GN0901],19860000
707986,707986,스치듯 안녕,[437],[윤종신],2662866,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,[GN0100],"[GN0105, GN0101]",20160120
707987,707987,숲의 빛,[729868],[Nature Piano],2221722,명상의 시간을 위한 뉴에이지 음악,[GN1800],"[GN1807, GN1801]",20131217


GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
           ...  
GN2900       뮤지컬
GN2901    세부장르전체
GN2902     국내뮤지컬
GN2903     국외뮤지컬
GN3000     크리스마스
Length: 254, dtype: object

,id,plylst_title,songs,tags,like_cnt,updt_date
0,61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",[락],71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[추억, 회상]",1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[까페, 잔잔한]",17,2017-08-28 07:09:34.000
3,147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",33,2019-12-05 15:15:18.000
4,27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",[댄스],9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",3,2020-04-17 04:31:11.000
115067,106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",[일렉],13,2015-12-24 17:23:19.000
115068,11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",4,2019-08-16 20:59:22.000
115069,131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...","[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",4,2019-10-25 23:40:42.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",[],1675,2019-05-27 14:14:33.000
1,131447,앨리스테이블,[],[],1,2014-07-16 15:24:24.000
2,51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",[],62,2008-06-21 23:26:22.000
3,45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",[],20,2017-10-30 18:15:43.000
4,79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",[],20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",[잔잔한],17,2015-12-17 14:06:05.000
23011,122127,,"[450275, 487671, 561031, 663944, 628672, 59121...","[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",10,2020-04-16 21:35:44.000
23012,77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",[],0,2019-03-27 15:27:40.000
23013,36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",[],31,2015-11-18 11:49:09.000


,id,plylst_title,songs,tags,like_cnt,updt_date
0,70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",[],6,2012-09-29 01:57:26.000
1,7461,,"[196298, 269984, 267805, 175867, 529244, 63825...","[나만의Best3, 인디아티스트들의추천음악]",0,2019-12-17 14:06:45.000
2,90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",[드라이브],21,2015-05-23 10:44:48.000
3,58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",[분위기],0,2019-03-14 09:47:34.000
4,102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",[],38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",[추억],16,2016-04-18 11:02:09.000
10736,936,,"[105140, 582252, 199262, 422915, 547967, 48791...","[띵곡의, 우울, 분위기, 드라이브, 산책]",1,2020-04-08 07:15:59.000
10737,110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",[기분전환],6,2016-06-29 00:57:21.000
10738,2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",[여름],4,2015-06-06 09:52:01.000


In [39]:
_index = ['{}{}'.format(i, j) for i in range(2) for j in range(0,10)]
_columns = [col[:-2] for col in genre.index if col[-2:]=="00"]
gnMap = pd.DataFrame(index=_index, columns=_columns)
for big in gnMap.columns:
    for small in gnMap.index:
        if big+small in genre.index:
            gnMap.loc[small, big] = genre[big+small]
gnMap = gnMap.iloc[:17, :]
gnMap

,GN01,GN02,GN03,GN04,GN05,GN06,GN07,GN08,GN09,GN10,...,GN21,GN22,GN23,GN24,GN25,GN26,GN27,GN28,GN29,GN30
00,발라드,댄스,랩/힙합,R&B/Soul,인디음악,록/메탈,성인가요,포크/블루스,POP,록/메탈,...,CCM,어린이/태교,종교음악,국악,아이돌,일렉트로니카,EDM,뮤직테라피,뮤지컬,크리스마스
01,세부장르전체,세부장르전체,세부장르전체,세부장르전체,세부장르전체,세부장르전체,세부장르전체,세부장르전체,세부장르전체,세부장르전체,...,세부장르전체,유아동요,가톨릭,세부장르전체,세부장르전체,세부장르전체,세부장르전체,세부장르전체,세부장르전체,NaN
02,'80,'80,랩 스타일,어반,포크,'70,신세대트로트,'60-'70,얼터너티브팝,모던록,...,국내CCM,창작동요,불교,크로스오버,남자 아이돌,댄스 스타일,하우스,ASMR/자연,국내뮤지컬,NaN
03,'90,'90,보컬 스타일,R&B,록,'80,전설의트로트,'80-'90,올디스,얼터너티브록,...,국외CCM,영어동요,NaN,국악가요,여자 아이돌,팝 스타일,클럽,힐링/명상/요가,국외뮤지컬,NaN
04,'00,'00,언더그라운드 힙합,NaN,일렉,'90,뽕짝트로트,'00,월드팝,프로그레시브/아트록,...,워십,영어동요,NaN,민요,랩/힙합,시대별,DJ,집중력,NaN,NaN
05,'10-,'10-,시대별,NaN,힙합,'00,트로트메들리,'10-,'60-'70,하드록,...,찬송가,영어동요,NaN,판소리/단가,발라드,NaN,NaN,숙면유도,NaN,NaN
06,NaN,NaN,NaN,NaN,발라드,'10-,트로트'60-'70,NaN,'80-'90,헤비메탈,...,성가,영어동화,NaN,풍물/사물놀이,댄스,NaN,NaN,반려동물,NaN,NaN
07,NaN,NaN,NaN,NaN,'90,NaN,트로트'80-'90,NaN,'00,뉴메탈/하드코어,...,연주곡,만화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
08,NaN,NaN,NaN,NaN,'00,NaN,트로트'00-,NaN,'10-,포스트록,...,어린이,자장가,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
09,NaN,NaN,NaN,NaN,'10-,NaN,성인가요'80-'90,NaN,NaN,'60,...,성경,태교동화,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# tags dictionary를 만들자.
def makeTagsSeries(which):
    tag_dict = {}
    for idx in which.index:
        temp = which.tags[idx]
        for _tag in temp:
            if _tag in tag_dict:
                tag_dict[_tag] += 1
            else:
                tag_dict[_tag] = 1
    return pd.Series(tag_dict).sort_values(ascending=False)
tagsTrain = makeTagsSeries(train)
tagsVal = makeTagsSeries(val)
tagsTest = makeTagsSeries(test)
display(tagsTrain) #, tagsVal, tagsTest)

기분전환      16465
감성        11417
휴식        11215
발라드       10796
잔잔한       10218
          ...  
사이드비          1
힙합전설          1
얀키            1
드렁크타이거        1
로맨틱쇼          1
Length: 29160, dtype: int64

기분전환       956
감성         688
드라이브       630
휴식         612
잔잔한        564
          ... 
섹시하게         1
산책하면서        1
탄탄한몸매        1
건강한몸         1
위로받고싶을때      1
Length: 4875, dtype: int64

기분전환       431
휴식         324
감성         314
드라이브       283
사랑         276
          ... 
오늘드디어컴백      1
얼터너티브힙합      1
크루           1
하드코어         1
드라마_OST      1
Length: 2824, dtype: int64

In [66]:
count = 0
which = test
for i, _ttl in tqdm(enumerate(which.plylst_title)):
    if _ttl == '':
        continue
    if _ttl != '':
        count += 1
        print(_ttl, which.songs[i], which.tags[i])
print("count: ", count) # 2041 / 10740 = 19%

노래방 안에서 [] ['스트레스']
봄바람과 듣고싶은 설레는 노래 [] ['사랑']
책읽으며 듣기좋은 잔잔 BGM♪ [] ['재즈', '기분전환', '뉴에이지']
한여름의 센티멘탈 피아노. [] []
매력있고 편안한 노래들입니다!♬ [] ['새벽']
조아요 누르고 또 조아요 누를뻔한 들을때마다 좋은곡(상시업뎃) [] ['Pop', '산책', '인디']
라붐유명한노래들-좋은노래 [] []
+안개낀 날씨와 어울리는 신비로운 Pop+ [] []
들으면 들을수록 더 좋아지는 발라드 [] []
사랑하는 나의 아버지 나의 맘 받으소서 [] []
기분전환! 음악과 함께하는 여행 [] []
봄기운 한가득 따뜻한 뉴에이지 [] ['봄', '연주곡', '피아노', '힐링', '설렘']
겨울날 따뜻한 핫팩처럼 주머니에 넣고다니고 싶은 음악 [] ['꿀잠', '겨울밤', '기분전환']
감성 자극 팝 음악 아는 노래부터 숨은 명곡까지 [] ['감성', '카페', '슬픔', '숨은', '팝']
오랫동안 알고싶은 피아노곡들 [] ['잔잔한']
마음뭉클해지는노래모음 [] ['감성', '슬픈노래', '보고싶어']
힘내요 우리 [] ['봄']
2015 음악 페스티벌 예습 1탄: 팝 & 가요 필청트랙!! [] ['산책']
살랑살랑 바람이 느껴지는 인디감성 가득한 노래 [] ['인디', '바람']
이별후. [] []
■SCENE ASMR : 호캉스 떠나는 직장인 B의 하루 [] ['호캉스', 'vlog', '포근한', '여름휴가', '편안한']
가수로만 알기 쉬운 싱어송라이터들 [] ['레전드', '발라드', '감성', '가요', '회상']
퍼렐 윌리엄스 프로듀싱 [] ['힙합', 'HipHop', '2000', 'PharrellWilliam']
들으면 아 하는 OST 명곡 [] ['매장', 'OST', '힐링', '영화']
sunny day...^^ [] ['설렘']
ㅋㅋㅋㅋ [] []
비트 오지고 조지는 랩&힙합 [] []
이어폰끼고 듣는 인디음악 [] ['이어폰끼고']
○ 추억감성  200

좋다ㅜㅜ [] []
Retro 80' [] ['레트로팝']
자기전에 이어폰 끼고 듣기 좋은 힙합 [] ['힙합']
차에서 크게 듣고 싶은 힙합~ [] ['랩']
상큼한 트렌디 POP ^_^ 취향저격 List :) [] []
뜨거운여름쿠바빛낭만 [] []
전설의 레전드는 죽지 않는다! (열거하기벅차추천수시업뎃!) [] ['메탈', '전설의락커들']
그 시절 등교 길을 책임 진 인디뮤직 [] ['산책', '추억']
봄 햇살과 함께 듣는 피아노 [] ['뉴에이지']
간주만 들어도 누구나 아! 는국민쏭 [] ['한국노래', '노래방애창곡', '간주게임']
2【꽁꽁】댄스『쾅『쾅75분♪ [] []
비오는날 네가생각나는날 [] []
빠운쓰 빠운쓰 추석맞이 본격 교통체증 극뽁송 [] ['추석', '취향저격DJ']
여름 휴가를 떠나자!! 여행떠나는 차안에서 듣기좋은 노래 [] ['배경음악', '감성', '여행', '휴가', '신나는']
내가듣는 발라드 데헷 [] ['잔잔한']
잘때들으면 잠잘오는 음악 [] ['감성', '밤']
마음이 편안해지는 음악#명상 [] ['오케스트라', '명상']
선선해진 이 밤, 이 노랜 어때? [] ['바람', '밤']
가벼운 산책할 때 듣기 좋은 음악 [] ['산책']
그리움이 가득한 6월의 피아노곡 [] ['여름']
★HIPHOP★ [] []
듣기만 해도 설레는 노래 모음 [] ['달달', '사랑']
tvtv [] []
분위기 있는 장소에 어울리는 노래 [] ['추억', '카페', '회상', '장르불문', '분위기']
비 온 뒤 무지개를 그리는 싱그러운 뉴에이지 [] []
마음을 안정시켜주는 뉴에이지. [] []
믿고듣는 국내힙합 & 알앤비 [] []
영혼이 치유되는 크로스오버 곡 [] []
밤이면 생각나는 잔잔한 노래. [] []
하루를 마무리하며 듣는 잔잔한 클래식 [] ['휴식']
◈울어도행복해..사랑하니까 [] []
방송노래폴더2 [] []
@런닝머신속도와 음악비트를 맞췄다! <6.0-7.0km구간> (전체재생) [] ['여름', '운동

그루브는 나의 6번 째 감각, 그 감각을 자극하는 Lounge Music [] ['애시드재즈', '딥하우스']
이 주의 Hidden Tracks : 11월 둘째 주! [] ['휴식']
늦은 밤, 분위기에 빠져드는 편하게 듣는 힙한 트랙 #7 [] []
#classic hiphop party mix [] ['골든에라', '90년대힙합']
허스키보이스~! [] []
내 귀 에  라 운 지. ( 실시간 업데이트 ) [] []
카페에서 듣기 적절한 노래 [] ['잔잔']
매력 넘치는 국내외 일렉 비트 총집합!! [] []
Country POP 모음집 [] []
★☆★☆[부드러운 힙합]☆★☆★ [] ['힙합']
잔잔한 감성노래 모음 [] ['밤', '가을']
더욱더 우울해지고 싶을때 듣는노래 [] ['눈물']
연말 파티에 어울리는 팝 [] ['연말', 'Pop', '신나는']
비트 좋은 외국힙합 모음 [] ['트랩', 'HipHop', '비트']
편안하게 해주는 CLASSIC곡들! [] []
감성발라드 모음// [] ['겨울', '휴식', '눈', '크리스마스']
벌써 12월, 아직도 썸만 타고 있다면 [] ['연말', '남친', '썸남썸녀', '연애', '고백']
일단 들어보시라~!신나지는 중독적인 팝° [] []
재즈 : 사랑스럽고 느끼한 재즈 음악 [] []
라디오 990 [] []
잠이오지 않는 밤, 나긋하게 나를 달래주는 lofi jazz hiphop [] ['휴식', '카페', '새벽', '숙면', '공부할때']
신나는 팝에 STEP 맞춰 야외 러닝타임! [] ['스포츠', '트랜디', 'running', '러닝', '신나는']
Billboard TheHot100 Top20 170909(미국차트) [] ['운동', 'TOP20', 'Billboard']
good [] []
나달렌 명곡 모아 듣자 ❣️ [] ['달샤벳', '뜨자_제발']
노래방애서 [] ['설렘', '사랑']
공부할 때 듣는 잔잔잔 BGM과 피아노곡들 [] []
눈이 동글! 아이의 집중력! [

대장나무 [] []
▶▶▶▶▶We Be Clubin' - HIP HOP [] ['힙합']
월요일을 이겨내자! [] []
'벚꽃엔딩' 없는 봄 플레이리스트 [] ['카페', '드라이브', '벚꽃']
목요일 [] []
나의 맘이 흥분되는 노래~! [] ['기분전환']
귀뿐 아니라 심장까지 녹을 것 같은 잔잔한 노래 [] []
찬송가 [] []
무더위, 비 를 기다리면서 이 노래 어때요 :)? [] ['출근길', '아침', '지하철', '기분전환']
집에서 뒹굴뒹굴~ 분위기 넘치는 방콕 R&B [] ['알앤비', '기분전환', '집순이', '편안한']
ZARA in 2015. MEL. [] ['기분전환']
어딘가 사랑이 뿜뿜뿜! 노래듣고 기분 up!  [] ['휴식', '운동', '댄스', '여행', '사랑']
마음이 편안해지는 J-jazz[2] [] ['까페']
Groovy Seoul [] ['랩']
여름에 듣기 좋은 노래! (국내곡 위주) [] []
방구석 섹시 플레이리스트 [] ['비오는날', '감성', '분위기', '센치']
이별에공감되는노래 [] ['잔잔한']
신나는 클럽Music 1 [] ['힙합']
리스트만 봐도 그시절 그때의 추억! [] ['회상']
봄날을 위한 감성인디음악♥ [] []
어느새 다가온 가을에 어울릴만한 노래들 [] ['감성', '아이돌', '잔잔한', '오마이걸_아린', '따뜻한']
듣고 또 듣는 노래 [] []
전투력 상승시키는 비장한 곡들 [] []
잔잔하게 시작 하고 싶을 때 듣는 노래:) [] []
락킹 하거나 그루빙 하거나 2 [] ['드라이브']
느긋하게, 재즈 [] []
멍하니 듣기좋은 연주곡들 [] []
봄비가 촉촉 내려와 276 [] ['봄비']
:74: [] []
Featuring good♥ [] ['랩']
신나서 저절로 들썩거리게 되는 노래 [] ['dance']
별빛이 내 머리 위로 살며시 다가온 밤의 노래 [] ['감성', '감성자극', '잔잔한', '위로', '따뜻한']
노래방 추천곡 (여자) [] ['휴식',

In [71]:
# common tags in val 3838 / 4875, test 2354 / 2824
# new tags
count = 0
for _tag in tagsTest.index:
    if _tag in tagsTrain:
        count += 1
count

2354

In [3]:
%%time
ko_model = fasttext.load_facebook_model('../../fasttext/cc.ko.300.bin')

NameError: name 'm_fasttext' is not defined

In [43]:
%%time
for w, sim in ko_model.wv.similar_by_word('페이커', 50):
    print(f'{w}: {sim}')

이상혁과: 0.6722542643547058
한왕호: 0.6251864433288574
이상혁을: 0.6240236759185791
이상혁의: 0.5952044129371643
Faker: 0.5852394104003906
이상혁이: 0.5729013681411743
이상혁은: 0.5607293844223022
이상혁: 0.5543851852416992
패킷만: 0.5511523485183716
롤챔스: 0.5419972538948059
허승훈: 0.5362598299980164
정글러: 0.5234872102737427
피넛: 0.5125508308410645
민호팀빠른: 0.5121454000473022
미드라이너: 0.5110136866569519
한왕호가: 0.5049764513969421
배성웅의: 0.5031503438949585
조일장: 0.49357157945632935
데프트: 0.4833090603351593
배성웅: 0.4822298288345337
벵기: 0.4799672067165375
오리아나를: 0.479117751121521
젠야타: 0.4703730642795563
그레이브즈: 0.4700821042060852
배준식: 0.46831583976745605
스멥: 0.4647432565689087
4SKT: 0.4634209871292114
8SKT: 0.4623197317123413
조성주와: 0.4616892337799072
한왕호의: 0.4615998864173889
그레이브즈가: 0.4603579342365265
faker: 0.4596383273601532
2세트에서: 0.45779842138290405
팀포트리스2: 0.45761021971702576
GuMiho: 0.4528508484363556
한왕호는: 0.45162147283554077
패패승승승: 0.451564759016037
이즈리얼: 0.45009690523147583
ROX는: 0.44970059394836426
로드호그: 0.449435561895370

In [16]:
api = KhaiiiApi()
morphses = []
def morphs(api, sentence):
    m = []
    print(sentence)
    for word in api.analyze(sentence):
        print(word)
        for morph in word.morphs:
            m.append((morph.lex, morph.tag))
    return m

# for i in train.index:
#     sentence = train.plylst_title[i]
#     morphs = []
#     print(f"{i}th playlist")
#     for word in api.analyze(sentence):
#         print(word)
#         for morph in word.morphs:
#             morphs.append((morph.lex, morph.tag))
#     morphses.append(morphs)
#     if i == 3:
#         break
morphses
morphs(api, "아버지가방에들어가신다.")

아버지가방에들어가신다.
아버지가방에들어가신다.	아버지/NNG + 가/JKS + 방/NNG + 에/JKB + 들어가/VV + 시/EP + ㄴ다/EF + ./SF


[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('시', 'EP'),
 ('ㄴ다', 'EF'),
 ('.', 'SF')]